In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
df = pd.read_csv('results/trans_results.csv')
perf_df = pd.read_csv('results/perfs.csv')

In [32]:
states = ["s", "d", "b", "f", "q"]
state_names = [s[0].upper() for s in states]

In [33]:
all_sessions = list(range(-4,19))
all_sessions.remove(0)
all_sessions

[-4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [34]:
trans_names = []
[[trans_names.append(ii+jj) for ii in state_names] for jj in state_names]
def label_expand(i):
    reg = np.zeros((1,25))
    reg[:,i] = 1
    reg_df = pd.DataFrame(reg.tolist(), columns=trans_names)
    return reg_df

rr = df['labels'].map(label_expand)
regression_df = pd.concat(rr.tolist())
regression_df['perf'] = df['session']
for ii in all_sessions:
    try:
        s = df['perf'][df['session'].iloc[ii]]
        regression_df['perf'].iloc[ii] = s
    except:
        regression_df['perf'].iloc[ii] = np.nan

regression_df = regression_df.dropna()

/tmp/ipykernel_2021576/2595843094.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_df['perf'].iloc[ii] = np.nan


In [35]:
from sklearn.linear_model import LinearRegression 
model=LinearRegression()
X = regression_df[trans_names]
Y = regression_df['perf']
model.fit(X,Y)
r_sq = model.score(X,Y)
r_sq
pd.DataFrame(model.coef_.reshape(1,25), columns=trans_names)

,SS,DS,BS,FS,QS,SD,DD,BD,FD,QD,...,SF,DF,BF,FF,QF,SQ,DQ,BQ,FQ,QQ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
session_mats = pd.read_csv("results/session_markovs.csv", index_col=0)
perfs = []
for s in all_sessions:
    if(s in perf_df["mission"].tolist()):
        p = [perf_df[perf_df['mission'] == s]['perf']][0].tolist()
        perfs = perfs + p
perfs = np.array(perfs)
session_mats['perfs'] = perfs

In [41]:
from sklearn.linear_model import LinearRegression 
import scipy.stats
model=LinearRegression()
X = session_mats[trans_names]
Y = session_mats['perfs']
model.fit(X,Y)
r_sq = model.score(X,Y)
pd.DataFrame(model.coef_.reshape(1,25), columns=trans_names)
nums = model.coef_
dist = scipy.stats.norm(nums.mean(), nums.std())
probs = [dist.cdf(n) for n in nums]
seg_df = pd.DataFrame(np.array(probs).reshape(1,25), columns=trans_names)
seg_df.to_csv("results/linear_reg_sig.csv")

In [40]:
seg_df.T > 0.8

,0
SS,False
DS,False
BS,True
FS,False
QS,False
SD,False
DD,False
BD,False
FD,False
QD,False
